In [2]:
import psycopg2 as ps
import pandas as pd
import altair as alt

Load data from database

In [160]:
conn = ps.connect(database="example", user="user", password="user_pass", host="picasso.experiments.cs.kuleuven.be", port="3895")
query = "SELECT * FROM actions"
df = pd.read_sql_query(query, conn)
conn.close()
#load data from csv
topX = pd.read_csv('avg_topX.csv')

#load data from csv
questionnaire = pd.read_csv('questionnaire_parsed.csv')

df = df.merge(questionnaire[['userid', 'isExpert', 'version']], on='userid', how='left')
df = df.drop(columns=['version_x'])
df = df.rename(columns={'version_y': 'version'})

#remove all actions from user 13456a187bb82a

df = df[df['userid'] != '41910f46383ae']


#set timestamp to 'DD-MM-YYYY HH:MM AM/PM', no seconds, or milliseconds

# df['formatted_tstamp'] = df['tstamp'].dt.strftime('%d-%m-%Y %I:%M %p')


#load data from csv


C:\Users\morti\AppData\Local\Temp\ipykernel_23216\488614218.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,id,userid,page,actionname,tstamp,json,isExpert,version
0,541,019132e398e18,index,Change weight,2023-05-14 09:00:12.369,"{'Axis': 'Pose', 'name': 'Change weight', 'pag...",Expert,LC + EH
1,542,019132e398e18,index,Change weight,2023-05-14 09:00:15.249,"{'Axis': 'Pose', 'name': 'Change weight', 'pag...",Expert,LC + EH
2,543,019132e398e18,index,Change weight,2023-05-14 09:00:17.038,"{'Axis': 'Pose', 'name': 'Change weight', 'pag...",Expert,LC + EH
3,139,f00db05d7655b,index,Change weight,2023-05-11 20:35:28.742,"{'Axis': 'Pose', 'name': 'Change weight', 'pag...",Non-Expert,LC + ES
4,140,f00db05d7655b,index,Change weight,2023-05-11 20:35:34.023,"{'Axis': 'Pose', 'name': 'Change weight', 'pag...",Non-Expert,LC + ES
...,...,...,...,...,...,...,...,...
3136,2682,3be8366dd8f36,ResultsPage,CloseDialog,2023-05-23 18:14:13.000,"{'name': 'CloseDialog', 'page': 'ResultsPage',...",Non-Expert,LC + EH
3137,2679,3be8366dd8f36,ResultsPage,CloseDialog,2023-05-23 18:14:15.000,"{'name': 'CloseDialog', 'page': 'ResultsPage',...",Non-Expert,LC + EH
3138,2680,3be8366dd8f36,ResultsPage,CloseDialog,2023-05-23 18:14:09.000,"{'name': 'CloseDialog', 'page': 'ResultsPage',...",Non-Expert,LC + EH
3139,2677,3be8366dd8f36,ResultsPage,openDialog,2023-05-23 18:14:14.000,"{'name': 'openDialog', 'page': 'ResultsPage', ...",Non-Expert,LC + EH


In [161]:
##table headers:id,userid,page,version,actionname,tstamp,json
#count times actionname 'openDialog' per user else set to 0
df_openDialog = df[df['actionname'] == 'openDialog'].groupby(['isExpert', 'userid']).count().reset_index()[['userid', 'actionname', 'isExpert']]
df_openDialog.columns = ['userid', 'count', 'isExpert']

#save to csv
df_openDialog.to_csv('countDialogOpen.csv', index=False)

df_openDialog

,userid,count,isExpert
0,2bf0b2caeede3,8,Expert
1,49eaabd33a20a,13,Expert
2,4a83d2a70ed38,25,Expert
3,5e09a989f3a0e8,39,Expert
4,66d235530e05e,33,Expert
5,79262589165e6,2,Expert
6,8a832fac71fbc,27,Expert
7,92541790deac9,29,Expert
8,99e22b182c01d,20,Expert
9,e22b6a3fc3f23,12,Expert


In [163]:
x = df[['page', 'actionname', 'isExpert', 'userid']]

x = x.groupby(['page', 'actionname', 'isExpert', 'userid']).size().reset_index(name='click_count')
x = x.groupby(['page', 'actionname', 'isExpert'])['click_count'].mean().reset_index()

#remove actionname = search rows
x = x[x['actionname'] != 'search']

y = x[x['page'] == 'AdvancedSettings']  # Filter for 'AdvancedSettings' page

#rename 'changed color' rows to 'color checkbox'
y.loc[y['actionname'] == 'Changed color', 'actionname'] = 'Color checkbox'
y.loc[y['actionname'] == 'Changed object', 'actionname'] = 'Object checkbox'
y.loc[y['actionname'] == 'Changed pose', 'actionname'] = 'Pose checkbox'

chart = alt.Chart(y).mark_bar().encode(
    x=alt.X('actionname:N', title='Action name', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('click_count:Q', title='Average click count'),
    color='isExpert:N',
    xOffset=alt.XOffset('isExpert:N', title='User type')
).properties(
    title='Average clickcount per action on the advanced Settings page',
    width=300
)

chart.save('charts/avg_clickcount_advancedSettings.png')

chart

alt.Chart(...)

In [164]:


# sum op sum of average number of clicks per expertise per page
total = x.groupby(['page', 'isExpert'])['click_count'].sum().reset_index()

#remove all page='ResultsPage' rows
total = total[total['page'] != 'ResultsPage']

#rename all rows with page='AdvancedSettings' to 'High control'
total.loc[total['page'] == 'AdvancedSettings', 'page'] = 'High control'

#rename all rows with page='index' to 'Low control'
total.loc[total['page'] == 'index', 'page'] = 'Low control'


chart = alt.Chart(total).mark_bar().encode(
    x=alt.X('page:N', title='Page',  axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('click_count:Q', title='Total number of clicks'),
    color='isExpert:N',
    xOffset='isExpert:N'
).properties(
    title='Total number of clicks per user type',
    width=200
)

chart.save('charts/total_clickcount.png')

chart

alt.Chart(...)

In [165]:
#duration of each user (see first and last action)

#first action
first = df.groupby(['userid', 'version','isExpert'])['tstamp'].min().reset_index()

# last action (actionname = 'submitTop10')
last = df[df['actionname'] == 'Submit Top 10'].groupby(['userid', 'version','isExpert'])['tstamp'].max().reset_index()

last2 = df.groupby(['userid', 'version','isExpert'])['tstamp'].max().reset_index()


#count how many times each user has clicked on 'Sumbit Top 10'
count = df[df['actionname'] == 'Submit Top 10'].groupby(['userid', 'version'])['actionname'].count().reset_index()

#merge count with first and last action

first = first.merge(count, on=['userid', 'version' ], how='left')


#merge first and last action

duration = first.merge(last, on=['userid', 'version', 'isExpert' ], how='left')



#if actionname = 'Submit Top 10' is NaN, set to 2
duration['actionname'] = duration['actionname'].fillna(1)


#calculate duration

duration['duration'] = duration['tstamp_y'] - duration['tstamp_x']

copy = duration.copy()

#devide duration by number of times user clicked on 'Submit Top 10'

duration = duration[duration['duration'] <= pd.Timedelta(minutes=20)]

duration = duration[duration['userid'] != '3be8366dd8f36'] #zero for some reason, brings down the whole average



duration['duration'] = duration['duration'].map(lambda x: x.total_seconds())


duration['duration'] = duration['duration'] / duration['actionname']

#calculate average duration per user type

duration = duration.groupby(['version','isExpert'])['duration'].mean().reset_index()

#plot


chart = alt.Chart(duration).mark_bar().encode(
    x=alt.X('version:N', title='User type', axis=alt.Axis(labelAngle=-45), sort=['LC + EH', 'LC + ES', 'HC + EH', 'HC + ES']),
    y=alt.Y('duration:Q', title='Average duration (seconds)'),
    color='isExpert',
    xOffset='isExpert'
).properties(
    title='Average session duration per version',
    width=200
)


chart

alt.Chart(...)

In [11]:
copy

,userid,version,isExpert,tstamp_x,actionname,tstamp_y,duration
0,019132e398e18,LC + EH,Expert,2023-05-14 09:00:12.369,1.0,NaT,NaT
1,04a419bf11412,LC + ES,Non-Expert,2023-05-12 06:52:13.035,4.0,2023-05-12 06:58:30.357,0 days 00:06:17.322000
2,0c215e7cea7f38,LC + EH,Non-Expert,2023-05-23 17:47:15.095,1.0,2023-05-23 17:51:12.767,0 days 00:03:57.672000
3,16f7882aa59908,HC + ES,Non-Expert,2023-05-21 19:04:06.860,2.0,2023-05-21 19:11:16.610,0 days 00:07:09.750000
4,29152f1697ff68,HC + EH,Non-Expert,2023-05-24 09:09:01.397,1.0,NaT,NaT
5,2bf0b2caeede3,LC + EH,Expert,2023-05-21 12:26:11.301,2.0,2023-05-21 12:36:07.075,0 days 00:09:55.774000
6,32993a5b624ce,HC + ES,Non-Expert,2023-05-25 14:46:57.118,1.0,2023-05-25 14:50:45.610,0 days 00:03:48.492000
7,3be8366dd8f36,LC + EH,Non-Expert,2023-05-23 18:00:00.000,1.0,2023-05-23 18:00:00.000,0 days 00:00:00
8,41910f46383ae,LC + ES,Non-Expert,2023-05-23 21:57:59.558,1.0,2023-05-23 21:59:55.097,0 days 00:01:55.539000
9,49eaabd33a20a,HC + ES,Expert,2023-05-17 08:31:43.909,1.0,2023-05-17 08:37:37.626,0 days 00:05:53.717000


In [166]:
#get all clicks from results page
clicks = df[df['page'] == 'ResultsPage']


clicks = clicks.groupby(['userid', 'version', 'isExpert'])['actionname'].count().reset_index().rename(columns={'actionname': 'clicks'})

#count times user clicked on 'Submit Top 10'

count = df[df['actionname'] == 'Submit Top 10'].groupby(['userid', 'version'])['actionname'].count().reset_index()


clicks = clicks.merge(count, on=['userid', 'version'], how='left')

clicks['actionname'] = clicks['actionname'].fillna(1)

clicks = clicks.rename(columns={'actionname': 'times_submit'})

clicks['clicks'] = clicks['clicks'] / clicks['times_submit']

x = clicks.copy()


#add LC-ES + HC-ES and LC-EH + HC-EH

clicks.loc[clicks['version'] == 'LC + ES', 'version'] = 'Explanations Shown'
clicks.loc[clicks['version'] == 'HC + ES', 'version'] = 'Explanations Shown'
clicks.loc[clicks['version'] == 'LC + EH', 'version'] = 'Explanations Hidden'
clicks.loc[clicks['version'] == 'HC + EH', 'version'] = 'Explanations Hidden'

clicks = clicks.groupby(['version', 'isExpert'])['clicks'].mean().reset_index()

chart = alt.Chart(clicks).mark_bar().encode(
    x=alt.X('version:N', title='Version', axis=alt.Axis(labelAngle=-45), sort=['LC + EH', 'LC + ES', 'HC + EH', 'HC + ES']),
    y=alt.Y('clicks:Q', title='Average number of clicks'),
    color='isExpert',
    xOffset='isExpert'

).properties(
    title='Average number of clicks on the results page per version',
    width=200
)

chart.save('charts/total_clicks_results.png')
chart

alt.Chart(...)

In [167]:
#list all actions on results page
actions = df[df['page'] == 'ResultsPage']

actions = actions.groupby(['userid','actionname', 'version', 'isExpert'])['actionname'].count().reset_index(name='click_count')

actions = actions.groupby(['actionname', 'version', 'isExpert'])['click_count'].mean().reset_index()

#remove the following actions

chart = alt.Chart(actions).mark_bar().encode(
    x=alt.X('actionname:N', title='Action', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('click_count:Q', title='Average number of clicks'),
    color='isExpert:N',
    xOffset='isExpert:N'
).properties(
    title='Average clickcount per action on the results page',
    width=300
)

chart.save('charts/average_clickcount_results.png')
chart

alt.Chart(...)

In [150]:
actions

,action,version,isExpert,click_count
0,CloseDialog,HC + EH,Expert,25.000000
1,CloseDialog,HC + EH,Non-Expert,16.333333
2,CloseDialog,HC + ES,Expert,8.666667
3,CloseDialog,HC + ES,Non-Expert,21.714286
4,CloseDialog,LC + EH,Expert,17.500000
5,CloseDialog,LC + EH,Non-Expert,8.571429
6,CloseDialog,LC + ES,Expert,36.500000
7,CloseDialog,LC + ES,Non-Expert,27.666667
8,New search,HC + EH,Expert,1.000000
9,New search,HC + EH,Non-Expert,1.000000


In [168]:
#list all actions

actions = df.groupby(['userid','actionname', 'version', 'isExpert'])['actionname'].count().reset_index(name='click_count')

actions = actions.groupby(['version', 'isExpert'])['click_count'].mean().reset_index()

chart = alt.Chart(actions).mark_bar().encode(
    x=alt.X('version:N', title='Action', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('click_count:Q', title='Average number of clicks'),
    color='isExpert:N',
    xOffset='isExpert:N'
).properties(
    title='Average clickcount per action',
    width=300
)

chart

alt.Chart(...)

In [122]:
x

,userid,version,isExpert,clicks,times_submit
0,019132e398e18,LC + EH,Expert,10.000000,1.0
1,04a419bf11412,LC + ES,Non-Expert,31.000000,4.0
2,0c215e7cea7f38,LC + EH,Non-Expert,28.000000,1.0
3,16f7882aa59908,HC + ES,Non-Expert,47.000000,2.0
4,29152f1697ff68,HC + EH,Non-Expert,72.000000,1.0
5,2bf0b2caeede3,LC + EH,Expert,24.000000,2.0
6,32993a5b624ce,HC + ES,Non-Expert,56.000000,1.0
7,3be8366dd8f36,LC + EH,Non-Expert,17.000000,1.0
8,41910f46383ae,LC + ES,Non-Expert,68.000000,1.0
9,49eaabd33a20a,HC + ES,Expert,53.000000,1.0
